# Gabor functions

Gabor functions have been shown to be good models for the behavior of simple cells of V1, the first layer of visual perception in the brain. Also, many machine learning models including Deep Learning models like CNNs and Transformers learn gabor functions in their first layer when applied to natural images.

This notebook lets you interactively change the functions parameters to quickly get an intuitive feel for them.

One way of expression a Gabor filter is $g(x,y; \theta, \sigma, \gamma, f, \phi, x_0, y_0) = e^{-\frac{x'^2 + \gamma^2 y'^2}{2\sigma^2}} \cos(2\pi f x' + \phi)$,

where

- $x' = (x - x_0) \cos(\theta) + (y - y_0) \sin(\theta)$
- $y' = -(x - x_0) \sin(\theta) + (y - y_0) \cos(\theta)$



In [22]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, Checkbox

In [23]:
def gabor(x, y, x0, y0, theta, sigma, f, phi, gamma):
    xp = (x - x0) * np.cos(theta) + (y - y0) * np.sin(theta)
    yp = -(x - x0) * np.sin(theta) + (y - y0) * np.cos(theta)
    envelope = np.exp(-(xp**2 + gamma**2 * yp**2) / (2 * sigma**2))
    carrier = np.cos(2 * np.pi * f * xp + phi)
    return envelope * carrier

In [24]:
def draw_sigma(x0, y0, sigma, theta):
    end_x = x0 + sigma * np.cos(theta)
    end_y = y0 + sigma * np.sin(theta)
    plt.arrow(x0, y0, end_x - x0, end_y - y0,
              head_width=0.5, head_length=0.5, fc='white', ec='white', length_includes_head=True)
    delta = 0.3
    plt.text(end_x, end_y + delta, r'$\sigma$', fontsize=12, color='white')

In [25]:
def plot_gabor(x0=0, y0=0, theta=0, sigma=1, f=0.5, phi=0, gamma=1, draw_parameters=False):
    x = np.linspace(-10, 10, 500)
    y = np.linspace(-10, 10, 500)
    X, Y = np.meshgrid(x, y)
    G = gabor(X, Y, x0, y0, theta, sigma, f, phi, gamma)

    plt.figure(figsize=(6, 6))
    plt.imshow(G, extent=(-10, 10, -10, 10), origin='lower')
    plt.colorbar()

    if draw_parameters:
        draw_sigma(x0, y0, sigma, theta)

    plt.show()

In [26]:
interactive_plot = interactive(plot_gabor,
                               x0=FloatSlider(min=-10, max=10, step=0.5, value=0, description='$x_0$:'),
                               y0=FloatSlider(min=-10, max=10, step=0.5, value=0, description='$y_0$:'),
                               theta=FloatSlider(min=0, max=np.pi, step=np.pi/16, value=0, description=r'$\theta$ (Theta):'),
                               sigma=FloatSlider(min=0.1, max=10, step=0.1, value=3, description=r'$\sigma$ (Sigma):'),
                               f=FloatSlider(min=0.1, max=1, step=0.05, value=0.5, description='$f$ (Freq.):'),
                               phi=FloatSlider(min=0, max=2*np.pi, step=np.pi/16, value=0, description=r'$\phi$ (Phi):'),
                               gamma=FloatSlider(min=0.1, max=10, step=0.1, value=1, description=r'$\gamma$ (Gamma):'),
                               draw_parameters=Checkbox(value=False, description='Draw parameters'))
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='$x_0$:', max=10.0, min=-10.0, step=0.5), FloatSlider…

# Using precision

We can also reformulate the Gabor function to use $\beta$ (precision) with $\beta=1/\sigma^2$ instead. Also, we can control the precision separately for $x$ and $y$.

In [27]:
def gabor(x, y, x0, y0, theta, beta_x, beta_y, alpha, f, phi):
    xp = (x - x0) * np.cos(theta) + (y - y0) * np.sin(theta)
    yp = -(x - x0) * np.sin(theta) + (y - y0) * np.cos(theta)
    envelope = alpha * np.exp(-0.5 * (beta_x * xp**2 + beta_y * yp**2))
    carrier = np.cos(2 * np.pi * f * xp + phi)
    return envelope * carrier

In [28]:
def draw_sigma(x0, y0, beta_x, beta_y, theta):
    sigma_x = 1 / np.sqrt(beta_x)
    sigma_y = 1 / np.sqrt(beta_y)

    # Calculate end points along rotated axes
    end_x_x = x0 + sigma_x * np.cos(theta)
    end_y_x = y0 + sigma_x * np.sin(theta)
    end_x_y = x0 - sigma_y * np.sin(theta)
    end_y_y = y0 + sigma_y * np.cos(theta)

    draw_labelled_arrow(x0, y0, end_x_x, end_y_x, r'$\sigma_x$')
    draw_labelled_arrow(x0, y0, end_x_y, end_y_y, r'$\sigma_y$')

def draw_labelled_arrow(x0, y0, x, y, label):
    plt.arrow(x0, y0, x - x0, y - y0,
              head_width=0.2, head_length=0.3, fc='white', ec='white', length_includes_head=True)
    delta = 0.3
    plt.text(x, y + delta, label, fontsize=12, color='white', horizontalalignment='center')

def plot_gabor(x0=0, y0=0, theta=0, beta_x=1, beta_y=1, alpha=1, f=0.5, phi=0, draw_parameters=False):
    x = np.linspace(-10, 10, 500)
    y = np.linspace(-10, 10, 500)
    X, Y = np.meshgrid(x, y)
    G = gabor(X, Y, x0, y0, theta, beta_x, beta_y, alpha, f, phi)

    plt.figure(figsize=(6, 6))
    plt.imshow(G, extent=(-10, 10, -10, 10), origin='lower')
    plt.colorbar()

    if draw_parameters:
        draw_sigma(x0, y0, beta_x, beta_y, theta)

    plt.show()

In [29]:
interactive_plot = interactive(plot_gabor,
                               x0=FloatSlider(min=-10, max=10, step=0.5, value=0, description='$x_0$:'),
                               y0=FloatSlider(min=-10, max=10, step=0.5, value=0, description='$y_0$:'),
                               theta=FloatSlider(min=0, max=np.pi, step=np.pi/16, value=0, description=r'$\theta$ (Theta):'),
                               beta_x=FloatSlider(min=0.01, max=10, step=0.05, value=0.1, description=r'$\beta_x$ (Beta x):'),
                               beta_y=FloatSlider(min=0.01, max=10, step=0.05, value=0.1, description=r'$\beta_y$ (Beta y):'),
                               alpha=FloatSlider(min=0.1, max=10, step=0.1, value=1, description=r'$\alpha$ (Alpha):'),
                               f=FloatSlider(min=0.1, max=1, step=0.05, value=0.5, description='$f$ (Freq.):'),
                               phi=FloatSlider(min=0, max=2*np.pi, step=np.pi/16, value=0, description=r'$\phi$ (Phi):'),
                               draw_parameters=Checkbox(value=False, description='Draw parameters'))
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='$x_0$:', max=10.0, min=-10.0, step=0.5), FloatSlider…